In [1]:
import numpy as np
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from skimage import io

np.random.seed(42)

In [2]:
def noisy(image, sigma):
    
    row,col= image.shape
    mean = 0
    img=np.array(image).astype(np.float32)
    gauss = np.random.normal(mean,sigma,(row,col))
    gauss = gauss.reshape(row,col)
    noisy = img + gauss
    return noisy

In [3]:
X = sorted(glob('/Volumes/Data/StarVoid/Sim/Raw/*.tif'))
Y = sorted(glob('/Volumes/Data/StarVoid/Sim/GT/*.tif'))

In [4]:
X = list(map(imread,X))
Y = list(map(imread,Y))
plt.subplot(121); plt.imshow(X[9],cmap='gray');   plt.axis('off'); plt.title('Raw image')

Text(0.5,1,'Raw image')

In [5]:
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_test = int(round(0.2*len(X)))
ind_pretrn, ind_test = ind[:-n_test], ind[-n_test:]
X_test, Y_test = [X[i] for i in ind_test] , [Y[i] for i in ind_test]
X_pretrn, Y_pretrn = [X[i] for i in ind_pretrn] , [Y[i] for i in ind_pretrn]
print('number of images: %3d' % len(X))
print('- training+validation:       %3d' % len(X_pretrn))
print('- test:     %3d' % len(X_test))

number of images: 150
- training+validation:       120
- test:      30


In [6]:
rng = np.random.RandomState(42)
ind = rng.permutation(len(X_pretrn))
n_val = int(round(0.15 * len(X_pretrn)))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X_pretrn[i] for i in ind_val]  , [Y_pretrn[i] for i in ind_val]
X_train, Y_train = [X_pretrn[i] for i in ind_train]  , [Y_pretrn[i] for i in ind_train]
print('number of images: %3d' % len(X))
print('- training:       %3d' % len(X_train))
print('- validation:     %3d' % len(X_val))

number of images: 150
- training:       102
- validation:      18


In [7]:
X_train = np.array(X_train)[:,:768,:736]
X_test = np.array(X_test)
X_val = np.array(X_val)[:,:768,:736]

Y_train = np.array(Y_train)[:,:768,:736]
Y_test = np.array(Y_test)
Y_val = np.array(Y_val)[:,:768,:736]

In [8]:
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/GTTrain.npy",Y_train)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/GTTest.npy",Y_test)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/GTVal.npy",Y_val)

In [9]:
i = 9
img, lbl = X_train[i], Y_train[i]
plt.figure(figsize=(16,10))
plt.subplot(121); plt.imshow(img,cmap='gray');   plt.axis('off'); plt.title('Raw image')
None;
print(np.min(img),np.max(img))

79 282


In [10]:
std=10.0
X_train10 = np.array([noisy(x,std) for x in tqdm(X_train)])
X_test10 = np.array([noisy(x,std) for x in tqdm(X_test)])
X_val10 = np.array([noisy(x,std) for x in tqdm(X_val)])

std=20.0
X_train20 = np.array([noisy(x,std) for x in tqdm(X_train)])
X_test20 = np.array([noisy(x,std) for x in tqdm(X_test)])
X_val20 = np.array([noisy(x,std) for x in tqdm(X_val)])

100%|██████████| 18/18 [00:00<00:00, 45.64it/s]


In [11]:
i = 9
img, lbl = X_train10[i], Y_train[i]
plt.figure(figsize=(16,10))
plt.subplot(121); plt.imshow(img,cmap='gray');   plt.axis('off'); plt.title('Raw image')
None;
print(np.min(img),np.max(img))

i = 9
img, lbl = X_train20[i], Y_train[i]
plt.figure(figsize=(16,10))
plt.subplot(121); plt.imshow(img,cmap='gray');   plt.axis('off'); plt.title('Raw image')
None;
print(np.min(img),np.max(img))

46.9542520202 290.110809422
3.25822037806 300.910062267


In [12]:
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/NoisyTrain.npy",X_train)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/NoisyTest.npy",X_test)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/NoisyVal.npy",X_val)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy10Train.npy",X_train10)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy10Test.npy",X_test10)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy10Val.npy",X_val10)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy20Train.npy",X_train20)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy20Test.npy",X_test20)
# np.save("/home/prakash/Desktop/NAS/StarVoid/Sim/NumpyData/Noisy20Val.npy",X_val20)


In [13]:
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainVal.npz", X_train=X_train, Y_train=Y_train, X_val=X_val, Y_val=Y_val)
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/Test.npz", X_test=X_test, Y_test=Y_test)

In [14]:

np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainVal10.npz",  X_train=X_train10, Y_train=Y_train, X_val=X_val10, Y_val=Y_val)
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/Test10.npz", X_test=X_test10, Y_test=Y_test)

In [15]:

np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainVal20.npz",  X_train=X_train20, Y_train=Y_train, X_val=X_val20, Y_val=Y_val)
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/Test20.npz", X_test=X_test20, Y_test=Y_test)

In [24]:
b=np.load('/Volumes/Data/StarVoid/Sim/NumpyData/TrainVal20.npz')

In [25]:
b['X_val'].shape

(18, 768, 736)

In [26]:
b['X_train'].shape

(102, 768, 736)

Noise2Void Data Prep

In [19]:
X_trainN2V=np.concatenate((X_train,X_test[:,:768,:736]))
X_valN2V=X_val
print(X_train10.shape)
print(X_test10.shape)

X_trainN2V10=np.concatenate((X_train10,X_test10[:,:768,:736]))
X_valN2V10=X_val10

X_trainN2V20=np.concatenate((X_train20,X_test20[:,:768,:736]))
X_valN2V20=X_val20

np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainValN2V.npz",  X_train=X_trainN2V, X_val=X_valN2V)
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainValN2V10.npz",  X_train=X_trainN2V10, X_val=X_valN2V10)
np.savez_compressed("/Volumes/Data/StarVoid/Sim/NumpyData/TrainValN2V20.npz",  X_train=X_trainN2V20, X_val=X_valN2V20)


(102, 768, 736)
(30, 773, 739)
